In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model

from art.utils import load_dataset

c:\Users\benke\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\benke\AppData\Local\Programs\Python\Python311\Lib\site-packages\art\estimators\certification\__init__.py:29: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


In [17]:
class Autoencoder(Model):
    def __init__(self, shape):
        super(Autoencoder, self).__init__()
        self.shape = shape
        self.encoder = tf.keras.Sequential([
            layers.Conv2D(112, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.MaxPool2D((2,2), padding='same'),
            layers.Conv2D(56, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2D(28, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2D(28, (3, 3), activation='leaky_relu', padding='same', strides=2)
        ])
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(28, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2DTranspose(28, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2DTranspose(56, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.UpSampling2D(size=(2,2), interpolation='nearest'),
            layers.Conv2DTranspose(112, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2D(3, kernel_size=(3,3), activation='sigmoid', padding='same')
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

shape = x_test.shape[1:]
autoencoder = Autoencoder(shape)

In [18]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/10


ValueError: in user code:

    File "c:\Users\benke\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\benke\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\benke\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\benke\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\benke\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    F